In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import datetime
from datetime import datetime
from datetime import timedelta

In [2]:
# Ignoring the warning related to Setting with copy.
pd.options.mode.chained_assignment = None

start_year = 2005

# Extract the current year
URL = 'https://www.resultados-futbol.com/ligue_1'
r = requests.get(URL)
page = r.content
soup = BeautifulSoup(page, 'html5lib')

current_year = int(soup.find('div', class_ = "titular-data").text.strip()[0:4])+1

# Define the countries and the years
Years = range(start_year,current_year)

Countries = ['Portugal','Spain','England','Italy','Germany','France']

# Create table_results_previous_years

In [3]:
%%time

# Create a dataframe with the extracted results of all teams for the 6 countries,
# for the last years

# Creation of an empty data frame for all teams in all countries, all years
table_results_previous_years = pd.DataFrame()

# 
for year in Years:
    
    print(f'Starting year {year}...')

    # Creation of an empty data frame for all teams in all countries, one year
    # Replace the name of the country by the name of the country/league in the
    # resultados-futbol website
    table_results_all_countries = pd.DataFrame()

    for Country in Countries:
        if Country == 'Portugal':
            country = 'portugal'
        elif Country == 'Spain':
            country = 'primera'
        elif Country == 'England':
            country = 'premier'
        elif Country == 'Italy':
            country = 'serie_a'
        elif Country == 'Germany':
            country = 'bundesliga'
        elif Country == 'France':
            country = 'ligue_1'
        else:
            print('Country not available, please choose between Portugal,\
        Spain, England, Italy, Germany or France')

        print(f'   Starting {Country} league of {year}...')
        

        # Per each country:


        # Get the URLs of the teams
        URL = f'https://www.resultados-futbol.com/{country}{year}'
        r = requests.get(URL)
        page = r.content
        soup = BeautifulSoup(page, 'html5lib')

        teams_URL_in_national_league = []

        for li in soup.find_all('li', class_ = "shield"):
            a = li.find('a', href=True)
            teams_URL_in_national_league.append(a['href'])



        # Creation of an empty data frame for all teams in one country, one year
        table_results_all_teams = pd.DataFrame()


        for team in teams_URL_in_national_league:
            
            URL = f'https://www.resultados-futbol.com{team}{year}'
            r = requests.get(URL)
            page = r.content
            soup = BeautifulSoup(page, 'html5lib')


            # Get the name of the competitions of each team
            list_of_competitions = []

            for div in soup.find_all('div', class_ = "title"):
                img = div.find('img', alt=True)
                list_of_competitions.append(img['alt'])

            # Get the name of the team (not team URL)
            div_class_team = soup.find('div', class_ = "name")
            div_class_team_a = div_class_team.find('a')
            Team = div_class_team_a.get_text()
            
            # Some teams have a space in the end of the name, we need to
            # delete it
            if Team[-1] == ' ':
                Team = Team[:-1]
            else:
                Team = Team


            # Creation of an empty data frame for all results of one team
            table_team = pd.DataFrame()

            # Creation of a data frame with all the results of the team,
            # including a column with the name of the competition
            page = pd.read_html(URL)
            
            # The table with the first competition results is the table
            # that appears in position 5:
            for competition_i in range(5,5+len(list_of_competitions)):
                table_competition_i = pd.DataFrame(page[competition_i])
                table_competition_i['Competition_original_name_URL'] = list_of_competitions[competition_i-5]
                table_team = pd.concat(
                    [table_team,table_competition_i],
                 ignore_index = True, axis = 0)


            # Concat the table created for one team with the empty dataframe
            table_results_all_teams = pd.concat(
                    [table_results_all_teams,
                     table_team],
                ignore_index = True, axis = 0)
        
            print(f'      {Team} {year} concluded')


        print(f'   {Country} league of {year} concluded')

        
        # Add the year to the table
        table_results_all_teams['Year'] = year
        

        # Concat the table created for one country with the empty dataframe
        table_results_all_countries = pd.concat(
                [table_results_all_countries,
                 table_results_all_teams],
             ignore_index = True, axis = 0)

    
    # Concat the table created for one year with the empty dataframe
    table_results_previous_years = pd.concat(
            [table_results_previous_years,
             table_results_all_countries],
         ignore_index = True, axis = 0)
    
    print(f'{year} concluded')
    print('\n')

table_results_previous_years

Starting year 2005...
   Starting Portugal league of 2005...
      Benfica 2005 concluded
      Porto 2005 concluded
      Sporting CP 2005 concluded
      Sporting Braga 2005 concluded
      Vitória Guimarães 2005 concluded
      Boavista 2005 concluded
      Marítimo 2005 concluded
      Rio Ave 2005 concluded
      Os Belenenses 2005 concluded
      Vitória Setúbal 2005 concluded
      Penafiel 2005 concluded
      Nacional 2005 concluded
      Gil Vicente 2005 concluded
      Académica 2005 concluded
      União de Leiria 2005 concluded
      Moreirense 2005 concluded
      Estoril 2005 concluded
      Beira Mar SC 2005 concluded
   Portugal league of 2005 concluded
   Starting Spain league of 2005...
      Barcelona 2005 concluded
      Real Madrid 2005 concluded
      Villarreal 2005 concluded
      Real Betis 2005 concluded
      Espanyol 2005 concluded
      Sevilla 2005 concluded
      Valencia 2005 concluded
      RC Deportivo 2005 concluded
      Athletic 2005 concluded
    

      Porto 2007 concluded
      Sporting CP 2007 concluded
      Benfica 2007 concluded
      Sporting Braga 2007 concluded
      Os Belenenses 2007 concluded
      Paços de Ferreira 2007 concluded
      União de Leiria 2007 concluded
      Nacional 2007 concluded
      Estrela Amadora 2007 concluded
      Boavista 2007 concluded
      Marítimo 2007 concluded
      Naval 2007 concluded
      Académica 2007 concluded
      Vitória Setúbal 2007 concluded
      Beira Mar SC 2007 concluded
      Desportivo Aves 2007 concluded
   Portugal league of 2007 concluded
   Starting Spain league of 2007...
      Real Madrid 2007 concluded
      Barcelona 2007 concluded
      Sevilla 2007 concluded
      Valencia 2007 concluded
      Villarreal 2007 concluded
      Real Zaragoza 2007 concluded
      Atlético 2007 concluded
      Recreativo 2007 concluded
      Getafe 2007 concluded
      Racing 2007 concluded
      Espanyol 2007 concluded
      Mallorca 2007 concluded
      RC Deportivo 2007 conclu

      Benfica 2009 concluded
      Nacional 2009 concluded
      Sporting Braga 2009 concluded
      Leixões 2009 concluded
      Académica 2009 concluded
      Vitória Guimarães 2009 concluded
      Marítimo 2009 concluded
      Paços de Ferreira 2009 concluded
      Estrela Amadora 2009 concluded
      Rio Ave 2009 concluded
      Naval 2009 concluded
      Vitória Setúbal 2009 concluded
      Os Belenenses 2009 concluded
      Trofense 2009 concluded
   Portugal league of 2009 concluded
   Starting Spain league of 2009...
      Barcelona 2009 concluded
      Real Madrid 2009 concluded
      Sevilla 2009 concluded
      Atlético 2009 concluded
      Villarreal 2009 concluded
      Valencia 2009 concluded
      RC Deportivo 2009 concluded
      Málaga 2009 concluded
      Mallorca 2009 concluded
      Espanyol 2009 concluded
      Almería 2009 concluded
      Racing 2009 concluded
      Athletic 2009 concluded
      Real Sporting 2009 concluded
      Osasuna 2009 concluded
      Real 

      Nacional 2011 concluded
      Paços de Ferreira 2011 concluded
      Rio Ave 2011 concluded
      Marítimo 2011 concluded
      União de Leiria 2011 concluded
      Olhanense 2011 concluded
      Vitória Setúbal 2011 concluded
      Beira Mar SC 2011 concluded
      Académica 2011 concluded
      Portimonense 2011 concluded
      Naval 2011 concluded
   Portugal league of 2011 concluded
   Starting Spain league of 2011...
      Barcelona 2011 concluded
      Real Madrid 2011 concluded
      Valencia 2011 concluded
      Villarreal 2011 concluded
      Sevilla 2011 concluded
      Athletic 2011 concluded
      Atlético 2011 concluded
      Espanyol 2011 concluded
      Osasuna 2011 concluded
      Real Sporting 2011 concluded
      Málaga 2011 concluded
      Racing 2011 concluded
      Real Zaragoza 2011 concluded
      Levante 2011 concluded
      R. Sociedad 2011 concluded
      Getafe 2011 concluded
      Mallorca 2011 concluded
      RC Deportivo 2011 concluded
      Hércules

      Nacional 2013 concluded
      Vitória Guimarães 2013 concluded
      Marítimo 2013 concluded
      Académica 2013 concluded
      Vitória Setúbal 2013 concluded
      Gil Vicente 2013 concluded
      Olhanense 2013 concluded
      Moreirense 2013 concluded
      Beira Mar SC 2013 concluded
   Portugal league of 2013 concluded
   Starting Spain league of 2013...
      Barcelona 2013 concluded
      Real Madrid 2013 concluded
      Atlético 2013 concluded
      R. Sociedad 2013 concluded
      Valencia 2013 concluded
      Málaga 2013 concluded
      Real Betis 2013 concluded
      Rayo Vallecano 2013 concluded
      Sevilla 2013 concluded
      Getafe 2013 concluded
      Levante 2013 concluded
      Athletic 2013 concluded
      Espanyol 2013 concluded
      Real Valladolid 2013 concluded
      Granada 2013 concluded
      Osasuna 2013 concluded
      Celta 2013 concluded
      Mallorca 2013 concluded
      RC Deportivo 2013 concluded
      Real Zaragoza 2013 concluded
   Spain l

      Marítimo 2015 concluded
      Rio Ave 2015 concluded
      Moreirense 2015 concluded
      Estoril 2015 concluded
      Boavista 2015 concluded
      Académica 2015 concluded
      Vitória Setúbal 2015 concluded
      Arouca 2015 concluded
      Gil Vicente 2015 concluded
      Penafiel 2015 concluded
   Portugal league of 2015 concluded
   Starting Spain league of 2015...
      Barcelona 2015 concluded
      Real Madrid 2015 concluded
      Atlético 2015 concluded
      Valencia 2015 concluded
      Sevilla 2015 concluded
      Villarreal 2015 concluded
      Athletic 2015 concluded
      Celta 2015 concluded
      Málaga 2015 concluded
      Espanyol 2015 concluded
      Rayo Vallecano 2015 concluded
      R. Sociedad 2015 concluded
      Elche 2015 concluded
      Levante 2015 concluded
      Getafe 2015 concluded
      RC Deportivo 2015 concluded
      Granada 2015 concluded
      Eibar 2015 concluded
      Almería 2015 concluded
      Córdoba 2015 concluded
   Spain league o

      Feirense 2017 concluded
      Boavista 2017 concluded
      Estoril 2017 concluded
      Chaves 2017 concluded
      Vitória Setúbal 2017 concluded
      Paços de Ferreira 2017 concluded
      Os Belenenses 2017 concluded
      Moreirense 2017 concluded
      Tondela 2017 concluded
      Arouca 2017 concluded
      Nacional 2017 concluded
   Portugal league of 2017 concluded
   Starting Spain league of 2017...
      Real Madrid 2017 concluded
      Barcelona 2017 concluded
      Atlético 2017 concluded
      Sevilla 2017 concluded
      Villarreal 2017 concluded
      R. Sociedad 2017 concluded
      Athletic 2017 concluded
      Espanyol 2017 concluded
      Alavés 2017 concluded
      Eibar 2017 concluded
      Málaga 2017 concluded
      Valencia 2017 concluded
      Celta 2017 concluded
      Las Palmas 2017 concluded
      Real Betis 2017 concluded
      RC Deportivo 2017 concluded
      Leganés 2017 concluded
      Real Sporting 2017 concluded
      Osasuna 2017 concluded
 

      Rio Ave 2019 concluded
      Boavista 2019 concluded
      Belenenses SAD 2019 concluded
      CD Santa Clara 2019 concluded
      Portimonense 2019 concluded
      Marítimo 2019 concluded
      Vitória Setúbal 2019 concluded
      Desportivo Aves 2019 concluded
      Tondela 2019 concluded
      Chaves 2019 concluded
      Nacional 2019 concluded
      Feirense 2019 concluded
   Portugal league of 2019 concluded
   Starting Spain league of 2019...
      Barcelona 2019 concluded
      Atlético 2019 concluded
      Real Madrid 2019 concluded
      Valencia 2019 concluded
      Getafe 2019 concluded
      Sevilla 2019 concluded
      Espanyol 2019 concluded
      Athletic 2019 concluded
      R. Sociedad 2019 concluded
      Real Betis 2019 concluded
      Alavés 2019 concluded
      Eibar 2019 concluded
      Leganés 2019 concluded
      Villarreal 2019 concluded
      Levante 2019 concluded
      Real Valladolid 2019 concluded
      Celta 2019 concluded
      Girona 2019 conclude

      Paços de Ferreira 2021 concluded
      CD Santa Clara 2021 concluded
      Vitória Guimarães 2021 concluded
      Moreirense 2021 concluded
      Famalicão 2021 concluded
      Belenenses SAD 2021 concluded
      Gil Vicente 2021 concluded
      Tondela 2021 concluded
      Boavista 2021 concluded
      Portimonense 2021 concluded
      Marítimo 2021 concluded
      Rio Ave 2021 concluded
      Farense 2021 concluded
      Nacional 2021 concluded
   Portugal league of 2021 concluded
   Starting Spain league of 2021...
      Atlético 2021 concluded
      Real Madrid 2021 concluded
      Barcelona 2021 concluded
      Sevilla 2021 concluded
      R. Sociedad 2021 concluded
      Real Betis 2021 concluded
      Villarreal 2021 concluded
      Celta 2021 concluded
      Granada 2021 concluded
      Athletic 2021 concluded
      Osasuna 2021 concluded
      Cádiz 2021 concluded
      Valencia 2021 concluded
      Levante 2021 concluded
      Getafe 2021 concluded
      Alavés 2021 con

,0,1,2,3,4,5,6,7,8,Competition_original_name_URL,Year
0,NaN,24 Feb 05,Finalizado,Benfica,1 - 1,CSKA Moskva,NaN,NaN,0,Europa League,2005
1,NaN,17 Feb 05,Finalizado,CSKA Moskva,2 - 0,Benfica,NaN,NaN,0,Europa League,2005
2,NaN,02 Dic 04,Finalizado,KSK Beveren,0 - 3,Benfica,NaN,NaN,0,Europa League,2005
3,NaN,25 Nov 04,Finalizado,Benfica,2 - 0,Dinamo Zagreb,NaN,NaN,0,Europa League,2005
4,NaN,04 Nov 04,Finalizado,Stuttgart,3 - 0,Benfica,NaN,NaN,0,Europa League,2005
...,...,...,...,...,...,...,...,...,...,...,...
93890,NaN,15 Ago 20,Aplazado,Dijon FCO,-,Nîmes,NaN,NaN,0,Partidos Amistosos,2021
93891,NaN,07 Ago 20,Finalizado,Dijon FCO,1 - 2,Metz,NaN,NaN,0,Partidos Amistosos,2021
93892,NaN,04 Ago 20,Aplazado,Dijon FCO,-,Strasbourg,NaN,NaN,1,Partidos Amistosos,2021
93893,NaN,01 Ago 20,Finalizado,Dijon FCO,0 - 0,Sochaux,NaN,NaN,0,Partidos Amistosos,2021


In [4]:
# Create an excel file with the table
table_results_previous_years.to_excel(
    'Table_results_previous_years_original.xlsx', index = False)

# Create functions to manage the table

In [5]:
def edit_raw_table(raw_table):


    # Drop duplicated games (the original table has all the games of all teams,
    # and as teams play against each other, many games are repeated).

    raw_table.drop_duplicates(inplace=True)

# Clean data, add columns, rename columns, reorder columns
    
    # Exclude all lines with NAs in column 2
    raw_table = raw_table[
        raw_table[2].notna()]
    
    # Exclude all columns with NA
    raw_table = raw_table.dropna(axis=1)
    
    # Rename columns
    raw_table.rename(columns = {1:'Date', 2:'Status', 3:'Home_team',
        4:'Result', 5:'Away_team'}, inplace = True)
    
    # Change the values of the match status
    raw_table['Status'] = raw_table['Status'].apply(
        lambda x: "Postponed" if "Aplazado" in x else \
            "Finalized" if "Finalizado" in x else \
            "Not played yet" if "en " in x else \
            "Still playing" if "'" in x else "Unknown")
    
    # Select only the columns we need
    raw_table = raw_table[[
        "Year","Date","Competition_original_name_URL",
        "Home_team","Away_team","Status","Result"]]

    
    # Create 2 columns with the goals of the home_team and away_team
    list_results = []
    
    for score in list(raw_table['Result']):
        if '-' in score:
            list_results.append(score)
        else:
            list_results.append('-')
    
    raw_table['Result'] = list_results
            
    list_home_score = []
    list_away_score = []
    
    for score in list(raw_table['Result']):
        if score == '-':
            list_home_score.append('-')
            list_away_score.append('-')
        elif '-' in score:
            list_home_score.append(score[:score.index('-')-1])
            list_away_score.append(score[score.index('-')+2:])
        else:
            list_home_score.append('-')
            list_away_score.append('-')
    
    raw_table['Home_score'] = list_home_score
    raw_table['Away_score'] = list_away_score
    
# Create a new column with the name of the country
    # First, we create lists with the names of the competitions per country
    raw_table['Country'] = list(raw_table['Competition_original_name_URL'])

    international_competition_list = ['Champions League', 'Europa League',
                                  'Copa Intercontinental', 'Copa Intertoto',
                                 'Mundial de Clubes', 'Supercopa Europa',
                                      'Previa Champions'
                                 ]

    portuguese_competitions = ['Liga Portuguesa', 
                               'Taça de Portugal', 
                               'Supercopa Portugal', 
                               'Copa de la Liga Portugal',
                              'Liga Portuguesa - Play Offs Ascenso']
    spanish_competitions = ['Primera División', 
                            'Copa del Rey', 
                            'Supercopa de España']
    english_competitions = ['Premier League', 
                            'FA Cup', 
                            'Community Shield', 
                            'EFL Cup']
    italian_competitions = ['Serie A', 
                            'Coppa Italia', 
                            'Supercopa de Italia']
    german_competitions = ['Bundesliga', 
                           'DFB Pokal', 
                           'Supercopa de Alemania', 
                           'Liga Pokal']
    french_competitions = ['Ligue 1', 
                           'Copa de Francia',  
                           'Supercopa Francia', 
                           'Copa de la Liga',
                          'Ligue 1 - Play Offs Ascenso']
    
    # Reset index before applying the following loop
    raw_table.reset_index(inplace = True)

    for i in range(len(list(raw_table['Country']))):
        if raw_table['Country'][i] in international_competition_list:
            raw_table['Country'][i] = 'International'
        elif raw_table['Country'][i] in portuguese_competitions:
            raw_table['Country'][i] = 'Portugal'
        elif raw_table['Country'][i] in spanish_competitions:
            raw_table['Country'][i] = 'Spain'
        elif raw_table['Country'][i] in english_competitions:
            raw_table['Country'][i] = 'England'
        elif raw_table['Country'][i] in italian_competitions:
            raw_table['Country'][i] = 'Italy'
        elif raw_table['Country'][i] in german_competitions:
            raw_table['Country'][i] = 'Germany'
        elif raw_table['Country'][i] in french_competitions:
            raw_table['Country'][i] = 'France'
        else:
            raw_table['Country'][i] = 'Friendly match'


    # Create dictionary to rename the competitions column

    competitions_initial_list = raw_table[
        'Competition_original_name_URL'].unique().tolist()

    dict_competitions = {}

    for initial_competition in competitions_initial_list:
        dict_competitions[initial_competition] = []

    for international_competition in international_competition_list:
        for initial_competition in competitions_initial_list:
            if international_competition in initial_competition:
                dict_competitions[initial_competition] = international_competition

    national_leagues_list = ['Bundesliga', 'Liga Portuguesa', 'Ligue 1',
                            'Premier League', 'Primera División', 'Serie A']

    for national_league in national_leagues_list:
        for initial_competition in competitions_initial_list:
            if national_league in initial_competition:
                dict_competitions[initial_competition] = 'National League'

    national_cups_list = ['Copa de Francia', 'Copa del Rey', 'Coppa Italia',
                            'FA Cup', 'DFB Pokal', 'Taça de Portugal']

    for national_cup in national_cups_list:
        for initial_competition in competitions_initial_list:
            if national_cup in initial_competition:
                dict_competitions[initial_competition] = 'National Cup'

    national_super_cup_list = ['Community Shield', 'Supercopa Francia', 'Supercopa Portugal',
                            'Supercopa de Alemania', 'Supercopa de Italia', 'Supercopa de España']

    for national_super_cup in national_super_cup_list:
        for initial_competition in competitions_initial_list:
            if national_super_cup in initial_competition:
                dict_competitions[initial_competition] = 'National Super Cup'

    national_leagues_cup_list = ['Copa de la Liga', 'EFL Cup', 'Liga Pokal']

    for national_leagues_cup in national_leagues_cup_list:
        for initial_competition in competitions_initial_list:
            if national_leagues_cup in initial_competition:
                dict_competitions[initial_competition] = 'National League Cup'
    
    # Convert the names of the remaining competitions in English
    
    dict_competitions['Trofeo Premier League Asia'] = 'Friendly match'            
    dict_competitions['Copa Intercontinental'] = 'Intercontinental Cup'
    dict_competitions['Copa Intertoto'] = 'Intertoto Cup'
    dict_competitions['Supercopa Europa'] = 'European Supercup'
    dict_competitions['Mundial de Clubes'] = 'Club World Cup'
    dict_competitions['Bundesliga - Play Offs Ascenso'] = 'Bundesliga Play Offs'
    dict_competitions['Liga Portuguesa - Play Offs Ascenso'] ='Liga Portuguesa Play Offs'
    dict_competitions['Ligue 1 - Play Offs Ascenso'] ='Ligue 1 Play Offs'

    # Still, there are competitions that have not been added to the dictionary:
    for competition in list(dict_competitions.keys()):
        if dict_competitions[competition] == 'Previa Champions':
            dict_competitions[competition] = 'Champions League'
    
    # For the ones that have not been added yet, we consider them as Friendly match
    for competition in list(dict_competitions.keys()):
        if dict_competitions[competition] == []:
            dict_competitions[competition] = 'Friendly match'


    # Add a column with the competition "standardized" name,
    # created with the dictionary above

    list_competition_standardized_name = []

    list_competition_original_name_URL = list(
        raw_table['Competition_original_name_URL'])

    for competition_original_name_URL in list_competition_original_name_URL:
        list_competition_standardized_name.append(
            dict_competitions[competition_original_name_URL])

    raw_table['Competition'] = list_competition_standardized_name

    raw_table = raw_table[
        ['Year', 'Country', 'Date',
            'Competition_original_name_URL', 'Competition', 'Home_team', 'Away_team', 
            'Status','Result', 'Home_score', 'Away_score']]
    
    
    # convert months in Spanish to months in English

    dict_months = {
        'Ene': 'Jan','Feb': 'Feb','Mar': 'Mar','Abr': 'Apr',
        'May': 'May','Jun': 'Jun','Jul': 'Jul','Ago': 'Aug',
        'Sep': 'Sep','Oct': 'Oct','Nov': 'Nov','Dic': 'Dec',
    }


    for i in range(len(list(raw_table['Date']))):
        month_es = raw_table['Date'][i][3:6]
        raw_table['Date'][i] = raw_table['Date'][i][0:3] +\
        dict_months[month_es] + raw_table['Date'][i][6:]
    

    # Create a new column with the date in the datetime format
    datetime_date_list = []

    for date in raw_table['Date']:
        date_object = datetime.strptime(date, "%d %b %y")
        datetime_date_list.append(date_object)
    
    raw_table.insert(list(raw_table.columns).index('Date')+1,'Datetime_date_list', datetime_date_list)
    
    
    # Drop results of non played games from previous seasons and reset index
    
    No_result = raw_table[(raw_table['Result'] == '-') &\
                          (raw_table['Year'] < current_year)
                         ].index
    raw_table.drop(No_result, inplace = True)
    raw_table.reset_index(inplace = True)


    # Add columns with 1x2 result and points for home and away team
    
    raw_table.loc[
        raw_table[
            'Home_score'] == raw_table['Away_score'], '1x2'] = 'x'
    raw_table.loc[
        raw_table[
            'Home_score'] > raw_table['Away_score'], '1x2'] = '1'
    raw_table.loc[
        raw_table[
            'Home_score'] < raw_table['Away_score'], '1x2'] = '2'
    
    raw_table['1x2'] = raw_table.apply(
    lambda row: row['1x2'] if row['Result'] != '-' else '-', axis=1)
    
    raw_table.loc[
        raw_table[
            '1x2'] == 'x', 'Points_Home_Team'] = int('1')
    raw_table.loc[
        raw_table[
            '1x2'] == '1', 'Points_Home_Team'] = int('3')
    raw_table.loc[
        raw_table[
            '1x2'] == '2', 'Points_Home_Team'] = int('0')
    raw_table.loc[
        raw_table[
            '1x2'] == 'x', 'Points_Away_Team'] = int('1')
    raw_table.loc[
        raw_table[
            '1x2'] == '1', 'Points_Away_Team'] = int('0')
    raw_table.loc[
        raw_table[
            '1x2'] == '2', 'Points_Away_Team'] = int('3')



    # Drop index column
    raw_table.drop(['index'], axis=1, inplace=True)
    
    return raw_table

In [6]:
def fatigue(raw_table):

    # Calculate the number of games played by each team in the last days.
    last_days = 21


    # Create a new column with the number of games played by the home_team
    # in the last days, excluding friendly matches.
    number_of_games_last_days_home_team_list = []

    for i in range(raw_table.shape[0]):
        Team = raw_table['Home_team'][i]
        match_day = raw_table['Datetime_date_list'][i]
        match_day_minus_last_days = match_day - timedelta(days = last_days)

        number_of_games_last_days_home_team_list.append(
            sum(
        (raw_table.Datetime_date_list >= match_day_minus_last_days) & 
        (raw_table.Datetime_date_list < match_day) & 
        ((raw_table.Home_team == Team) | (raw_table.Away_team == Team)) & 
        (raw_table.Competition != 'Friendly match')
        )
        )

    raw_table['number_of_games_last_days_home_team'] = number_of_games_last_days_home_team_list

    # Create a new column with the number of games played by the away_team
    # in the last days, excluding friendly matches.
    number_of_games_last_days_away_team_list = []

    for i in range(raw_table.shape[0]):
        Team = raw_table['Away_team'][i]
        match_day = raw_table['Datetime_date_list'][i]
        match_day_minus_last_days = match_day - timedelta(days = last_days)

        number_of_games_last_days_away_team_list.append(
            sum(
        (raw_table.Datetime_date_list >= match_day_minus_last_days) & 
        (raw_table.Datetime_date_list < match_day) & 
        ((raw_table.Away_team == Team) | (raw_table.Home_team == Team)) & 
        (raw_table.Competition != 'Friendly match')
        )
        )

    raw_table['number_of_games_last_days_away_team'] = number_of_games_last_days_away_team_list

    # Note: teams outside the 6 national leagues will appear in international
    # games with 0 or very little games in the last days. However this is not
    # a problem, as we are not predicting game results of their national leagues,
    # nor international competitions.
    
    # Exclude all competitions which are not national leagues, which is the only type
    # of competition we will make predictions
    raw_table = raw_table[raw_table['Competition'] == 'National League']
    
    # Re-set index after having dropped many rows
    raw_table.reset_index(drop=True,inplace=True)
    
    return raw_table

In [7]:
def points_respective_year_and_last_games(raw_table):
    
    # Calculate the number of points that each team has in the respective
    # season and also in the last games.
    
    n_last_games = 5
    
    # Create new columns that show the number of points of the home_team
    # and away_team
    
    number_of_points_respective_year_home_team_list = []
    number_of_points_last_games_home_team_list = []
    
    number_of_points_respective_year_away_team_list = []
    number_of_points_last_games_away_team_list = []
    
    
    for i in range(raw_table.shape[0]):
        
        # First, let's start with the points accumulated by the home_team
        # in the respective year and in the last games.

        home_team = raw_table.loc[i, 'Home_team']

        df_aux_respective_year_home_team = raw_table[((raw_table['Home_team'] == home_team) |\
            (raw_table['Away_team'] == home_team)) &\
            (raw_table['Year'] == raw_table.loc[i, 'Year']) &\
            (raw_table['Datetime_date_list'] < raw_table.loc[i, 'Datetime_date_list']) &\
            (raw_table['Competition'] == 'National League')
                          ]
        
        df_aux_last_games_home_team = df_aux_respective_year_home_team.sort_values(
            'Datetime_date_list')[-n_last_games:]
          
        if df_aux_last_games_home_team.shape[0] > 0:
            
            Points_respective_year_home_team = \
            (df_aux_respective_year_home_team[df_aux_respective_year_home_team['Home_team'] == home_team]\
             ['Points_Home_Team'].sum() +\
            df_aux_respective_year_home_team[df_aux_respective_year_home_team['Away_team'] == home_team]\
             ['Points_Away_Team'].sum())        
            
            Points_last_games_home_team = \
            (df_aux_last_games_home_team[df_aux_last_games_home_team['Home_team'] == home_team]\
             ['Points_Home_Team'].sum() +\
            df_aux_last_games_home_team[df_aux_last_games_home_team['Away_team'] == home_team]\
             ['Points_Away_Team'].sum())
        
        else:
            Points_respective_year_home_team = 0
            Points_last_games_home_team = 0
           
        number_of_points_respective_year_home_team_list.append(Points_respective_year_home_team)
        number_of_points_last_games_home_team_list.append(Points_last_games_home_team)
        
        
        # Now, let's calculate how many points the away_team has accumulated
        # in the respective year and in the last games
        
        away_team = raw_table.loc[i, 'Away_team']
        
        df_aux_respective_year_away_team = raw_table[((raw_table['Home_team'] == away_team) |\
            (raw_table['Away_team'] == away_team)) &\
            (raw_table['Year'] == raw_table.loc[i, 'Year']) &\
            (raw_table['Datetime_date_list'] < raw_table.loc[i, 'Datetime_date_list']) &\
            (raw_table['Competition'] == 'National League')
                                                 ]
                                                  
        df_aux_last_games_away_team = df_aux_respective_year_away_team.sort_values(
            'Datetime_date_list')[-n_last_games:]                             
                                                  
        if df_aux_last_games_away_team.shape[0] > 0:
            
            Points_respective_year_away_team = \
            (df_aux_respective_year_away_team[df_aux_respective_year_away_team['Home_team'] == away_team]\
             ['Points_Home_Team'].sum() +\
            df_aux_respective_year_away_team[df_aux_respective_year_away_team['Away_team'] == away_team]\
             ['Points_Away_Team'].sum())
            
            Points_last_games_away_team = \
            (df_aux_last_games_away_team[df_aux_last_games_away_team['Home_team'] == away_team]\
             ['Points_Home_Team'].sum() +\
            df_aux_last_games_away_team[df_aux_last_games_away_team['Away_team'] == away_team]\
             ['Points_Away_Team'].sum())
        
        else:
            Points_respective_year_away_team = 0
            Points_last_games_away_team = 0

        number_of_points_respective_year_away_team_list.append(Points_respective_year_away_team)
        number_of_points_last_games_away_team_list.append(Points_last_games_away_team)
        
    raw_table['points_respective_year_home_team'] = number_of_points_respective_year_home_team_list
    raw_table['points_respective_year_away_team'] = number_of_points_respective_year_away_team_list
    
    raw_table['points_last_games_home_team'] = number_of_points_last_games_home_team_list
    raw_table['points_last_games_away_team'] = number_of_points_last_games_away_team_list


    # We are only counting the points for the National League competitions,
    # as we are not predicting game results of other competitions.

    return raw_table

In [8]:
def points_between_teams(raw_table):

    # Calculate the number of points that each team has got in the
    # same match in the previous years.

    
    # Create new columns that show the number of points of the home_team
    # and away_team for the same match in the previous years
    
    number_of_points_between_teams_home_team_list = []
    
    number_of_points_between_teams_away_team_list = []
    
    for i in range(raw_table.shape[0]):
        
        home_team = raw_table.loc[i, 'Home_team']
        away_team = raw_table.loc[i, 'Away_team']

        df_aux = raw_table[((raw_table['Home_team'] == home_team) &\
            (raw_table['Away_team'] == away_team)) &\
            (raw_table['Datetime_date_list'] < raw_table.loc[i, 'Datetime_date_list']) &\
            (raw_table['Competition'] == 'National League')
                          ]
        
        if df_aux.shape[0] > 0:
            
            Points_home_team = \
            df_aux[df_aux['Home_team'] == home_team]['Points_Home_Team'].sum()
            
            Points_away_team = \
            df_aux[df_aux['Away_team'] == away_team]['Points_Away_Team'].sum()
        
        else:
            Points_home_team = 0
            
            Points_away_team = 0
            
        number_of_points_between_teams_home_team_list.append(Points_home_team)
            
        number_of_points_between_teams_away_team_list.append(Points_away_team)
        
        
        
    raw_table['points_between_teams_home_team'] = number_of_points_between_teams_home_team_list
        
    raw_table['points_between_teams_away_team'] = number_of_points_between_teams_away_team_list


    # We are only counting the points for the National League competitions,
    # as we are not predicting game results of other competitions.

    return raw_table

# Apply the functions to the table_results_previous_years

In [9]:
table_results_previous_years = pd.read_excel(
    'Table_results_previous_years_original.xlsx')

In [10]:
%%time
table_results_previous_years = edit_raw_table(table_results_previous_years)
table_results_previous_years

CPU times: user 1min 9s, sys: 774 ms, total: 1min 10s
Wall time: 1min 11s


,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Status,Result,Home_score,Away_score,1x2,Points_Home_Team,Points_Away_Team
0,2005,International,24 Feb 05,2005-02-24,Europa League,Europa League,Benfica,CSKA Moskva,Finalized,1 - 1,1,1,x,1.0,1.0
1,2005,International,17 Feb 05,2005-02-17,Europa League,Europa League,CSKA Moskva,Benfica,Finalized,2 - 0,2,0,1,3.0,0.0
2,2005,International,02 Dec 04,2004-12-02,Europa League,Europa League,KSK Beveren,Benfica,Finalized,0 - 3,0,3,2,0.0,3.0
3,2005,International,25 Nov 04,2004-11-25,Europa League,Europa League,Benfica,Dinamo Zagreb,Finalized,2 - 0,2,0,1,3.0,0.0
4,2005,International,04 Nov 04,2004-11-04,Europa League,Europa League,Stuttgart,Benfica,Finalized,3 - 0,3,0,1,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52384,2021,France,23 Dec 20,2020-12-23,Ligue 1,National League,Nîmes,Dijon FCO,Finalized,1 - 3,1,3,2,0.0,3.0
52385,2021,Friendly match,04 Aug 20,2020-08-04,Partidos Amistosos,Friendly match,Rodez,Nîmes,Finalized,1 - 1,1,1,x,1.0,1.0
52386,2021,Friendly match,31 Jul 20,2020-07-31,Partidos Amistosos,Friendly match,Grenoble,Nîmes,Finalized,2 - 0,2,0,1,3.0,0.0
52387,2021,Friendly match,01 Aug 20,2020-08-01,Partidos Amistosos,Friendly match,Dijon FCO,Sochaux,Finalized,0 - 0,0,0,x,1.0,1.0


In [11]:
%%time
table_results_previous_years = fatigue(table_results_previous_years)
table_results_previous_years

CPU times: user 28min 9s, sys: 9.04 s, total: 28min 18s
Wall time: 28min 35s


,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Status,Result,Home_score,Away_score,1x2,Points_Home_Team,Points_Away_Team,number_of_games_last_days_home_team,number_of_games_last_days_away_team
0,2005,Portugal,22 May 05,2005-05-22,Liga Portuguesa,National League,Boavista,Benfica,Finalized,1 - 1,1,1,x,1.0,1.0,2,2
1,2005,Portugal,14 May 05,2005-05-14,Liga Portuguesa,National League,Benfica,Sporting CP,Finalized,1 - 0,1,0,1,3.0,0.0,3,5
2,2005,Portugal,07 May 05,2005-05-07,Liga Portuguesa,National League,Penafiel,Benfica,Finalized,1 - 0,1,0,1,3.0,0.0,3,4
3,2005,Portugal,30 Apr 05,2005-04-30,Liga Portuguesa,National League,Benfica,Os Belenenses,Finalized,1 - 0,1,0,1,3.0,0.0,4,3
4,2005,Portugal,24 Apr 05,2005-04-24,Liga Portuguesa,National League,Estoril,Benfica,Finalized,1 - 2,1,2,2,0.0,3.0,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35612,2021,France,28 Feb 21,2021-02-28,Ligue 1,National League,Nîmes,Nantes,Finalized,1 - 1,1,1,x,1.0,1.0,5,4
35613,2021,France,13 Dec 20,2020-12-13,Ligue 1,National League,Nantes,Dijon FCO,Finalized,1 - 1,1,1,x,1.0,1.0,4,3
35614,2021,France,30 Aug 20,2020-08-30,Ligue 1,National League,Nantes,Nîmes,Finalized,2 - 1,2,1,1,3.0,0.0,1,1
35615,2021,France,14 Feb 21,2021-02-14,Ligue 1,National League,Dijon FCO,Nîmes,Finalized,0 - 2,0,2,2,0.0,3.0,6,4


In [12]:
%%time
table_results_previous_years = points_respective_year_and_last_games(table_results_previous_years)
table_results_previous_years

CPU times: user 14min, sys: 4.67 s, total: 14min 5s
Wall time: 14min 12s


,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Status,Result,...,Away_score,1x2,Points_Home_Team,Points_Away_Team,number_of_games_last_days_home_team,number_of_games_last_days_away_team,points_respective_year_home_team,points_respective_year_away_team,points_last_games_home_team,points_last_games_away_team
0,2005,Portugal,22 May 05,2005-05-22,Liga Portuguesa,National League,Boavista,Benfica,Finalized,1 - 1,...,1,x,1.0,1.0,2,2,49.0,64.0,1.0,10.0
1,2005,Portugal,14 May 05,2005-05-14,Liga Portuguesa,National League,Benfica,Sporting CP,Finalized,1 - 0,...,0,1,3.0,0.0,3,5,61.0,61.0,7.0,13.0
2,2005,Portugal,07 May 05,2005-05-07,Liga Portuguesa,National League,Penafiel,Benfica,Finalized,1 - 0,...,0,1,3.0,0.0,3,4,37.0,61.0,9.0,10.0
3,2005,Portugal,30 Apr 05,2005-04-30,Liga Portuguesa,National League,Benfica,Os Belenenses,Finalized,1 - 0,...,0,1,3.0,0.0,4,3,58.0,42.0,10.0,10.0
4,2005,Portugal,24 Apr 05,2005-04-24,Liga Portuguesa,National League,Estoril,Benfica,Finalized,1 - 2,...,2,2,0.0,3.0,3,4,26.0,55.0,3.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35612,2021,France,28 Feb 21,2021-02-28,Ligue 1,National League,Nîmes,Nantes,Finalized,1 - 1,...,1,x,1.0,1.0,5,4,24.0,23.0,9.0,5.0
35613,2021,France,13 Dec 20,2020-12-13,Ligue 1,National League,Nantes,Dijon FCO,Finalized,1 - 1,...,1,x,1.0,1.0,4,3,13.0,8.0,5.0,6.0
35614,2021,France,30 Aug 20,2020-08-30,Ligue 1,National League,Nantes,Nîmes,Finalized,2 - 1,...,1,1,3.0,0.0,1,1,1.0,3.0,1.0,3.0
35615,2021,France,14 Feb 21,2021-02-14,Ligue 1,National League,Dijon FCO,Nîmes,Finalized,0 - 2,...,2,2,0.0,3.0,6,4,15.0,15.0,1.0,3.0


In [13]:
table_results_previous_years.to_excel(
    'Table_results_previous_years_edited.xlsx', index = False)

# Create table_results_current_year

In [14]:
%%time

# Create a dataframe with the extracted results of all teams for the 6 countries,
# for the current year


# Creation of an empty data frame for all teams in all countries
table_results_current_year = pd.DataFrame()

# Replace the name of the country by the name of the country/league in the
# resultados-futbol website
for Country in Countries:
    if Country == 'Portugal':
        country = 'portugal'
    elif Country == 'Spain':
        country = 'primera'
    elif Country == 'England':
        country = 'premier'
    elif Country == 'Italy':
        country = 'serie_a'
    elif Country == 'Germany':
        country = 'bundesliga'
    elif Country == 'France':
        country = 'ligue_1'
    else:
        print('Country not available, please choose between Portugal,\
    Spain, England, Italy, Germany or France')

    print(f'   Starting {Country} league of {current_year}...')


    # Per each country:


    # Get the URLs of the teams
    URL = f'https://www.resultados-futbol.com/{country}'
    r = requests.get(URL)
    page = r.content
    soup = BeautifulSoup(page, 'html5lib')

    teams_URL_in_national_league = []

    for li in soup.find_all('li', class_ = "shield"):
        a = li.find('a', href=True)
        teams_URL_in_national_league.append(a['href'])


    # Creation of an empty data frame for all teams in one country, one year
    table_all_teams = pd.DataFrame()


    for team in teams_URL_in_national_league:

        URL = f'https://www.resultados-futbol.com/partidos/{team}'
        r = requests.get(URL)
        page = r.content
        soup = BeautifulSoup(page, 'html5lib')


        #  Get the name of the competitions of each team
        list_of_competitions = []

        for div in soup.find_all('div', class_ = "title"):
            img = div.find('img', alt=True)
            list_of_competitions.append(img['alt'])

        # Get the name of the team (not team URL)
        div_class_team = soup.find('div', class_ = "name")
        div_class_team_a = div_class_team.find('a')
        Team = div_class_team_a.get_text()

        # Some teams have a space in the end of the name, we need to
        # delete it
        if Team[-1] == ' ':
            Team = Team[:-1]
        else:
            Team = Team


        # Creation of an empty data frame for all results of one team
        table_team = pd.DataFrame()

        # Creation of data frame all the results of the team,
        # including a column with the name of the competition
        page = pd.read_html(URL)

        # The table with the first competition results is the table
        # that appears in position 5:
        for competition_i in range(5,5+len(list_of_competitions)):
            table_competition_i = pd.DataFrame(page[competition_i])
            table_competition_i['Competition_original_name_URL'] = list_of_competitions[competition_i-5]
            table_team = pd.concat(
                [table_team,table_competition_i],
             ignore_index = True, axis = 0)


        # Concat the table created for one team with the empty dataframe
        table_all_teams = pd.concat(
                [table_all_teams,
                 table_team],
            ignore_index = True, axis = 0)

        print(f'      {Team} concluded')


    print(f'   {Country} league concluded')


    table_all_teams['Year'] = current_year


    # Concat the table created for one country with the empty dataframe
    table_results_current_year = pd.concat(
            [table_results_current_year,
             table_all_teams],
         ignore_index = True, axis = 0)


print(f'{current_year} concluded')
print('\n')

table_results_current_year

   Starting Portugal league of 2022...
      Porto concluded
      Sporting CP concluded
      Benfica concluded
      Sporting Braga concluded
      Gil Vicente concluded
      Vitória Guimarães concluded
      Estoril concluded
      Portimonense concluded
      Marítimo concluded
      CD Santa Clara concluded
      Vizela concluded
      Paços de Ferreira concluded
      Boavista concluded
      Tondela concluded
      Moreirense concluded
      Arouca concluded
      Famalicão concluded
      Belenenses SAD concluded
   Portugal league concluded
   Starting Spain league of 2022...
      Real Madrid concluded
      Sevilla concluded
      Real Betis concluded
      Barcelona concluded
      Atlético concluded
      Villarreal concluded
      R. Sociedad concluded
      Athletic concluded
      Rayo Vallecano concluded
      Celta concluded
      Valencia concluded
      Osasuna concluded
      Espanyol concluded
      Elche concluded
      Getafe concluded
      Granada concluded
 

,0,1,2,3,4,5,6,7,8,Competition_original_name_URL,Year
0,NaN,07 Dic 21,Finalizado,Porto,1 - 3,Atlético,NaN,NaN,1576,Champions League Grupo 2,2022
1,NaN,24 Nov 21,Finalizado,Liverpool,2 - 0,Porto,NaN,NaN,550,Champions League Grupo 2,2022
2,NaN,03 Nov 21,Finalizado,Milan,1 - 1,Porto,NaN,NaN,1413,Champions League Grupo 2,2022
3,NaN,19 Oct 21,Finalizado,Porto,1 - 0,Milan,NaN,NaN,758,Champions League Grupo 2,2022
4,NaN,28 Sep 21,Finalizado,Porto,1 - 5,Liverpool,NaN,NaN,257,Champions League Grupo 2,2022
...,...,...,...,...,...,...,...,...,...,...,...
5688,NaN,24 Jul 21,Finalizado,Saint-Étienne,2 - 3,Clermont,NaN,NaN,0,Partidos Amistosos,2022
5689,NaN,21 Jul 21,Aplazado,Saint-Étienne,-,Nice,NaN,NaN,0,Partidos Amistosos,2022
5690,NaN,17 Jul 21,Finalizado,Saint-Étienne,2 - 1,Grenoble,NaN,NaN,10,Partidos Amistosos,2022
5691,NaN,16 Jul 21,Finalizado,Saint-Étienne,1 - 1,Bourg-Péronnas,NaN,NaN,8,Partidos Amistosos,2022


In [15]:
# Create an excel file with the final table
table_results_current_year.to_excel(
    'Table_results_current_year_original.xlsx', index = False)

# Apply functions to table_results_current_year

In [16]:
table_results_current_year = pd.read_excel('Table_results_current_year_original.xlsx')

In [17]:
%%time
table_results_current_year = edit_raw_table(table_results_current_year)
table_results_current_year

CPU times: user 2.17 s, sys: 15.1 ms, total: 2.18 s
Wall time: 2.2 s


,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Status,Result,Home_score,Away_score,1x2,Points_Home_Team,Points_Away_Team
0,2022,Friendly match,07 Dec 21,2021-12-07,Champions League Grupo 2,Champions League,Porto,Atlético,Finalized,1 - 3,1,3,2,0.0,3.0
1,2022,Friendly match,24 Nov 21,2021-11-24,Champions League Grupo 2,Champions League,Liverpool,Porto,Finalized,2 - 0,2,0,1,3.0,0.0
2,2022,Friendly match,03 Nov 21,2021-11-03,Champions League Grupo 2,Champions League,Milan,Porto,Finalized,1 - 1,1,1,x,1.0,1.0
3,2022,Friendly match,19 Oct 21,2021-10-19,Champions League Grupo 2,Champions League,Porto,Milan,Finalized,1 - 0,1,0,1,3.0,0.0
4,2022,Friendly match,28 Sep 21,2021-09-28,Champions League Grupo 2,Champions League,Porto,Liverpool,Finalized,1 - 5,1,5,2,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,2022,France,02 Jan 22,2022-01-02,Copa de Francia,National Cup,Jura Sud,Saint-Étienne,Finalized,1 - 4,1,4,2,0.0,3.0
3256,2022,France,19 Dec 21,2021-12-19,Copa de Francia,National Cup,Lyon-Duchère,Saint-Étienne,Finalized,0 - 1,0,1,2,0.0,3.0
3257,2022,Friendly match,17 Jul 21,2021-07-17,Partidos Amistosos,Friendly match,Saint-Étienne,Grenoble,Finalized,2 - 1,2,1,1,3.0,0.0
3258,2022,Friendly match,16 Jul 21,2021-07-16,Partidos Amistosos,Friendly match,Saint-Étienne,Bourg-Péronnas,Finalized,1 - 1,1,1,x,1.0,1.0


In [18]:
%%time
table_results_current_year = fatigue(table_results_current_year)
table_results_current_year

CPU times: user 14.1 s, sys: 89.6 ms, total: 14.2 s
Wall time: 14.4 s


,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Status,Result,Home_score,Away_score,1x2,Points_Home_Team,Points_Away_Team,number_of_games_last_days_home_team,number_of_games_last_days_away_team
0,2022,Portugal,15 May 22,2022-05-15,Liga Portuguesa,National League,Porto,Estoril,Not played yet,-,-,-,-,NaN,NaN,3,3
1,2022,Portugal,08 May 22,2022-05-08,Liga Portuguesa,National League,Benfica,Porto,Not played yet,-,-,-,-,NaN,NaN,2,3
2,2022,Portugal,30 Apr 22,2022-04-30,Liga Portuguesa,National League,Porto,Vizela,Not played yet,-,-,-,-,NaN,NaN,4,3
3,2022,Portugal,24 Apr 22,2022-04-24,Liga Portuguesa,National League,Sporting Braga,Porto,Not played yet,-,-,-,-,NaN,NaN,3,4
4,2022,Portugal,16 Apr 22,2022-04-16,Liga Portuguesa,National League,Porto,Portimonense,Not played yet,-,-,-,-,NaN,NaN,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,2022,France,06 Mar 22,2022-03-06,Ligue 1,National League,Saint-Étienne,Metz,Not played yet,-,-,-,-,NaN,NaN,3,3
2129,2022,France,21 Nov 21,2021-11-21,Ligue 1,National League,Metz,Girondins Bordeaux,Finalized,3 - 3,3,3,x,1.0,1.0,1,2
2130,2022,France,30 Oct 21,2021-10-30,Ligue 1,National League,Metz,Saint-Étienne,Finalized,1 - 1,1,1,x,1.0,1.0,2,2
2131,2022,France,20 Apr 22,2022-04-20,Ligue 1,National League,Girondins Bordeaux,Saint-Étienne,Not played yet,-,-,-,-,NaN,NaN,3,3


In [19]:
%%time
table_results_current_year = points_respective_year_and_last_games(table_results_current_year)
table_results_current_year

CPU times: user 21.8 s, sys: 155 ms, total: 22 s
Wall time: 22.3 s


,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Status,Result,...,Away_score,1x2,Points_Home_Team,Points_Away_Team,number_of_games_last_days_home_team,number_of_games_last_days_away_team,points_respective_year_home_team,points_respective_year_away_team,points_last_games_home_team,points_last_games_away_team
0,2022,Portugal,15 May 22,2022-05-15,Liga Portuguesa,National League,Porto,Estoril,Not played yet,-,...,-,-,NaN,NaN,3,3,59.0,27.0,0.0,0.0
1,2022,Portugal,08 May 22,2022-05-08,Liga Portuguesa,National League,Benfica,Porto,Not played yet,-,...,-,-,NaN,NaN,2,3,47.0,59.0,0.0,0.0
2,2022,Portugal,30 Apr 22,2022-04-30,Liga Portuguesa,National League,Porto,Vizela,Not played yet,-,...,-,-,NaN,NaN,4,3,59.0,23.0,0.0,0.0
3,2022,Portugal,24 Apr 22,2022-04-24,Liga Portuguesa,National League,Sporting Braga,Porto,Not played yet,-,...,-,-,NaN,NaN,3,4,38.0,59.0,0.0,0.0
4,2022,Portugal,16 Apr 22,2022-04-16,Liga Portuguesa,National League,Porto,Portimonense,Not played yet,-,...,-,-,NaN,NaN,2,2,59.0,26.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,2022,France,06 Mar 22,2022-03-06,Ligue 1,National League,Saint-Étienne,Metz,Not played yet,-,...,-,-,NaN,NaN,3,3,18.0,20.0,6.0,1.0
2129,2022,France,21 Nov 21,2021-11-21,Ligue 1,National League,Metz,Girondins Bordeaux,Finalized,3 - 3,...,3,x,1.0,1.0,1,2,8.0,12.0,2.0,5.0
2130,2022,France,30 Oct 21,2021-10-30,Ligue 1,National League,Metz,Saint-Étienne,Finalized,1 - 1,...,1,x,1.0,1.0,2,2,6.0,5.0,3.0,2.0
2131,2022,France,20 Apr 22,2022-04-20,Ligue 1,National League,Girondins Bordeaux,Saint-Étienne,Not played yet,-,...,-,-,NaN,NaN,3,3,20.0,18.0,0.0,0.0


# Correct Team names of table_results_current_year

In [20]:
# Create 2 lists (one for previous years and another for the current year)
# with the unique names of the teams that have played the national leagues.

list_teams_previous_years = []
list1 = list_teams_previous_years

list_teams_current_year = []
list2 = list_teams_current_year


list1 = list(set(list(table_results_previous_years['Home_team']) +\
                 list(table_results_previous_years['Away_team'])))

list2 = list(set(list(table_results_current_year['Home_team']) +\
                 list(table_results_current_year['Away_team'])))

df_match = pd.DataFrame({'Name current year': list2})

df_match

,Name current year
0,Arsenal
1,Stade Rennais
2,Hertha BSC
3,Porto
4,B. Leverkusen
...,...
111,Clermont
112,Alavés
113,Roma
114,Sevilla


In [21]:
# Use the library fuzzywuzzy to match the names in both lists. The library
# gives us a score from 0 to 100: the higher the score, the more similar the
# names are. First I create a column with the name and the score of the 2
# top teams most similar, and then a second column only with the score of the
# most similar name.

df_match['Suggested from previous years'] = df_match['Name current year'].apply(
        (lambda x: process.extract(x, list1)[0:2])
    )

df_match['Suggested from previous years Score'] = df_match['Name current year'].apply(
        (lambda x: process.extractOne(x, list1)[1])
    )

df_match.sort_values(by = 'Suggested from previous years Score').head(20)

,Name current year,Suggested from previous years,Suggested from previous years Score
68,Brentford,"[(Watford, 62), (Benevento, 56)]",62
111,Clermont,"[(Palermo FC, 64), (Levante, 53)]",64
82,Vizela,"[(Aston Villa, 66), (Sevilla, 62)]",66
47,Venezia,"[(Valencia, 67), (Spezia, 62)]",67
17,Salernitana,"[(Lens, 68), (SPAL, 68)]",68
9,Brighton Hove Alb.,"[(Brighton & Hove Albion, 87), (Torino, 60)]",87
87,Vitória,"[(Vitória Guimarães, 90), (Vitória Setúbal, 90)]",90
14,Olympique,"[(Olympique Lyonnais, 90), (Olympique Marseill...",90
44,Eintracht,"[(Eintracht Frankfurt, 90), (Inter, 72)]",90
18,Paços Ferreira,"[(Paços de Ferreira, 95), (SC Freiburg, 58)]",95


In [22]:
# The first 5 teams (Brentford, Clermont, Vizela, Venezia, Salernitana)
# are suggested names from other teams as they appear in the table for
# the first time in 2022 (it is their first appearence in the most important
# national league since the start year of this project). However, the
# following 6 teams are the ones with similar names in previous years.
# Those (and only those) are the names we need to correct.

# For Vitória and Olympique we get the same score for the 1st and 2nd suggestion.
# I've checked manually what is the right one.

table_results_current_year.loc[(
    table_results_current_year['Home_team'] == 'Brighton Hove Alb.'), 'Home_team'] = 'Brighton & Hove Albion'
table_results_current_year.loc[(
    table_results_current_year['Away_team'] == 'Brighton Hove Alb.'), 'Away_team'] = 'Brighton & Hove Albion'

table_results_current_year.loc[(
    table_results_current_year['Home_team'] == 'Vitória'), 'Home_team'] = 'Vitória Guimarães'
table_results_current_year.loc[(
    table_results_current_year['Away_team'] == 'Vitória'), 'Away_team'] = 'Vitória Guimarães'

table_results_current_year.loc[(
    table_results_current_year['Home_team'] == 'Olympique'), 'Home_team'] = 'Olympique Marseille'
table_results_current_year.loc[(
    table_results_current_year['Away_team'] == 'Olympique'), 'Away_team'] = 'Olympique Marseille'

table_results_current_year.loc[(
    table_results_current_year['Home_team'] == 'Eintracht'), 'Home_team'] = 'Eintracht Frankfurt'
table_results_current_year.loc[(
    table_results_current_year['Away_team'] == 'Eintracht'), 'Away_team'] = 'Eintracht Frankfurt'

table_results_current_year.loc[(
    table_results_current_year['Home_team'] == 'Mönchengladbach'), 'Home_team'] = 'B. Mönchengladbach'
table_results_current_year.loc[(
    table_results_current_year['Away_team'] == 'Mönchengladbach'), 'Away_team'] = 'B. Mönchengladbach'

table_results_current_year.loc[(
    table_results_current_year['Home_team'] == 'Paços Ferreira'), 'Home_team'] = 'Paços de Ferreira'
table_results_current_year.loc[(
    table_results_current_year['Away_team'] == 'Paços Ferreira'), 'Away_team'] = 'Paços de Ferreira'

In [23]:
table_results_current_year.to_excel(
    'Table_results_current_year_edited.xlsx', index = False)

# Join Tables previous years and current year

In [24]:
table_results_previous_years = pd.read_excel('Table_results_previous_years_edited.xlsx')
table_results_current_year = pd.read_excel('Table_results_current_year_edited.xlsx')

In [25]:
table_results_all_years = pd.concat(
            [table_results_previous_years,
             table_results_current_year],
         ignore_index = True, axis = 0)

In [26]:
table_results_all_years

,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Status,Result,...,Away_score,1x2,Points_Home_Team,Points_Away_Team,number_of_games_last_days_home_team,number_of_games_last_days_away_team,points_respective_year_home_team,points_respective_year_away_team,points_last_games_home_team,points_last_games_away_team
0,2005,Portugal,22 May 05,2005-05-22,Liga Portuguesa,National League,Boavista,Benfica,Finalized,1 - 1,...,1,x,1.0,1.0,2,2,49,64,1,10
1,2005,Portugal,14 May 05,2005-05-14,Liga Portuguesa,National League,Benfica,Sporting CP,Finalized,1 - 0,...,0,1,3.0,0.0,3,5,61,61,7,13
2,2005,Portugal,07 May 05,2005-05-07,Liga Portuguesa,National League,Penafiel,Benfica,Finalized,1 - 0,...,0,1,3.0,0.0,3,4,37,61,9,10
3,2005,Portugal,30 Apr 05,2005-04-30,Liga Portuguesa,National League,Benfica,Os Belenenses,Finalized,1 - 0,...,0,1,3.0,0.0,4,3,58,42,10,10
4,2005,Portugal,24 Apr 05,2005-04-24,Liga Portuguesa,National League,Estoril,Benfica,Finalized,1 - 2,...,2,2,0.0,3.0,3,4,26,55,3,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37745,2022,France,06 Mar 22,2022-03-06,Ligue 1,National League,Saint-Étienne,Metz,Not played yet,-,...,-,-,NaN,NaN,3,3,18,20,6,1
37746,2022,France,21 Nov 21,2021-11-21,Ligue 1,National League,Metz,Girondins Bordeaux,Finalized,3 - 3,...,3,x,1.0,1.0,1,2,8,12,2,5
37747,2022,France,30 Oct 21,2021-10-30,Ligue 1,National League,Metz,Saint-Étienne,Finalized,1 - 1,...,1,x,1.0,1.0,2,2,6,5,3,2
37748,2022,France,20 Apr 22,2022-04-20,Ligue 1,National League,Girondins Bordeaux,Saint-Étienne,Not played yet,-,...,-,-,NaN,NaN,3,3,20,18,0,0


# Apply function points_between_teams

In [27]:
%%time
table_results_all_years = points_between_teams(table_results_all_years)
table_results_all_years

CPU times: user 6min 26s, sys: 2.1 s, total: 6min 28s
Wall time: 6min 31s


,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Status,Result,...,Points_Home_Team,Points_Away_Team,number_of_games_last_days_home_team,number_of_games_last_days_away_team,points_respective_year_home_team,points_respective_year_away_team,points_last_games_home_team,points_last_games_away_team,points_between_teams_home_team,points_between_teams_away_team
0,2005,Portugal,22 May 05,2005-05-22,Liga Portuguesa,National League,Boavista,Benfica,Finalized,1 - 1,...,1.0,1.0,2,2,49,64,1,10,0.0,0.0
1,2005,Portugal,14 May 05,2005-05-14,Liga Portuguesa,National League,Benfica,Sporting CP,Finalized,1 - 0,...,3.0,0.0,3,5,61,61,7,13,0.0,0.0
2,2005,Portugal,07 May 05,2005-05-07,Liga Portuguesa,National League,Penafiel,Benfica,Finalized,1 - 0,...,3.0,0.0,3,4,37,61,9,10,0.0,0.0
3,2005,Portugal,30 Apr 05,2005-04-30,Liga Portuguesa,National League,Benfica,Os Belenenses,Finalized,1 - 0,...,3.0,0.0,4,3,58,42,10,10,0.0,0.0
4,2005,Portugal,24 Apr 05,2005-04-24,Liga Portuguesa,National League,Estoril,Benfica,Finalized,1 - 2,...,0.0,3.0,3,4,26,55,3,10,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37745,2022,France,06 Mar 22,2022-03-06,Ligue 1,National League,Saint-Étienne,Metz,Not played yet,-,...,NaN,NaN,3,3,18,20,6,1,17.0,5.0
37746,2022,France,21 Nov 21,2021-11-21,Ligue 1,National League,Metz,Girondins Bordeaux,Finalized,3 - 3,...,1.0,1.0,1,2,8,12,2,5,3.0,18.0
37747,2022,France,30 Oct 21,2021-10-30,Ligue 1,National League,Metz,Saint-Étienne,Finalized,1 - 1,...,1.0,1.0,2,2,6,5,3,2,11.0,11.0
37748,2022,France,20 Apr 22,2022-04-20,Ligue 1,National League,Girondins Bordeaux,Saint-Étienne,Not played yet,-,...,NaN,NaN,3,3,20,18,0,0,33.0,15.0


In [28]:
table_results_all_years.to_excel(
    'Table_results_all_years_edited.xlsx', index = False)